In [1]:
import numpy as np

In [2]:
import csv

In [3]:
import os

In [4]:
to_save_dir = '../../dgl/apps/kg/data/wiki/'

In [5]:
raw_dir = '../data/raw/'

In [6]:
!head ../data/raw/wikidata_20190805.qpq_item_statements.csv

source_item_id,edge_property_id,target_item_id,el_rank
31,1344,1088364,1
31,1151,3247091,1
31,1546,1308013,1
31,5125,7112200,1
31,38,4916,0
31,38,232415,1
31,1792,7021332,1
31,2852,1061257,1
31,2852,25648793,1


In [7]:
entity_input_file = raw_dir + 'wikidata_20190805.item.csv'
entity_output_file = to_save_dir + 'entities.dict'
entity_keep_cols = [0, 3]
entity_text_format = "{}\t{}\n"
entity_skip_lines = 1
entity_dict = {}

In [8]:
!head {entity_input_file}

id,en_label,en_description,enwiki_title
51475818,YouTube as a source of information on kidney stone disease.,scientific article published on 4 December 2010,
51475821,The sinus lift with phycogenic bone substitute. A histomorphometric study.,scientific article published in June 2005,
51475829,Economic aspects of single-tooth replacement.,scientific article published in June 2005,
51475835,"Template:Peace, Unity, and Development Party/meta/shortname",,"Template:Peace, Unity, and Development Party/meta/shortname"
51475865,Long-term results and survival rate of implants treated with guided bone regeneration: a 5-year case series prospective study.,scientific article published in June 2005,
51475886,Flora Virgiliana,Bologna : a spese dell' autore (Tipografia Mareggiani) 1869 | Biblioteca Digital del Real Jardin Botanico de Madrid View Book (External),
51475902,Atrial fibrillation: can we cure it if we can't explain it?,scientific article published in May 2005,
51475904,,,
51475909,,,


In [9]:
edge_input_file = raw_dir + 'wikidata_20190805.property.csv'
edge_output_file = to_save_dir + 'relations.dict'
edge_keep_cols = [0, 1]
edge_text_format = "{}\t{}\n"
edge_skip_lines = 1 
edge_dict = {}

In [10]:
def write_to_file_with_format_and_dict(input_file, output_file, keep_cols, text_format, skip_lines = 0, dictionary = None):
    m = len(keep_cols)
    
    i = 0
    reverse_dict = {}
    with open(input_file, 'r') as file_inp, open(output_file, 'w') as file_out:
        inp = csv.reader(file_inp)
        for _ in range(skip_lines):
            next(inp)
        for l in inp:
            to_write = [l[col] for col in keep_cols if l[col]]
            original_key = int(to_write[0])
            to_write[0] = i
            if len(to_write) < m:
                continue
            i += 1
            text = to_write[1]
            #if text not in reverse_dict:
            #reverse_dict[text] = original_key
            dictionary[original_key] = text
    return reverse_dict

In [11]:
edge_mapping_dict = write_to_file_with_format_and_dict(
    input_file=edge_input_file,
    output_file=edge_output_file,
    keep_cols=edge_keep_cols,
    text_format=edge_text_format,
    skip_lines=edge_skip_lines,
    dictionary=edge_dict,
)

In [12]:
entity_mapping_dict = write_to_file_with_format_and_dict(
    input_file=entity_input_file,
    output_file=entity_output_file,
    keep_cols=entity_keep_cols,
    text_format=entity_text_format,
    skip_lines=entity_skip_lines,
    dictionary=entity_dict,
)

In [13]:
np.random.seed(0)

In [17]:
def write_to_file_with_format_for_triplet(input_file, output_file, output_file2, output_file3, text_format,\
                                          skip_lines = 0, triplet_head_idx = 0, triplet_tail_idx = 2, \
                                         triplet_edge_idx = 1):
    with open(input_file, 'r') as file_inp, open(output_file, 'w') as file_out,\
    open(output_file2, 'w') as file_out2, open(output_file3, 'w') as file_out3:
        
        inp = csv.reader(file_inp)
        for _ in range(skip_lines):
            next(inp)
        for l in inp:
            to_write = []
            triplet_head, triplet_edge, triplet_tail = \
            int(l[triplet_head_idx]), int(l[triplet_edge_idx]), int(l[triplet_tail_idx])
#             if not triplet_head in entity_mapping_dict or not triplet_tail in entity_mapping_dict or \
#             not triplet_edge in edge_mapping_dict:
#             if (not triplet_head in entity_dict and not triplet_tail in entity_dict):
#                 continue
            if entity_dict.get(triplet_head, None) is None and entity_dict.get(triplet_tail, None) is None:
                continue
            if triplet_head not in entity_dict:
                entity_dict[triplet_head] = None
            if triplet_tail not in entity_dict:
                entity_dict[triplet_tail] = None
            if triplet_edge not in edge_dict:
                edge_dict[triplet_edge] = None
            rand_int = np.random.randint(200)
            if rand_int > 1:
                write_file_out = file_out
            elif rand_int == 0:
                write_file_out = file_out2
            else:
                write_file_out = file_out3
            write_file_out.write(text_format.format(triplet_head, triplet_edge, triplet_tail))
        return None, None

In [18]:
triplet_input_file = raw_dir + 'wikidata_20190805.qpq_item_statements.csv'
triplet_output_file = to_save_dir + 'train.txt'
triplet_output_file_2 = to_save_dir + 'valid.txt'
triplet_output_file_3 = to_save_dir + 'test.txt'
triplet_text_format = "{}\t{}\t{}\n"
triplet_skip_lines = 1

In [19]:
final_ent_dict, final_rel_dict = write_to_file_with_format_for_triplet(
    input_file=triplet_input_file,
    output_file=triplet_output_file,
    output_file2=triplet_output_file_2,
    output_file3=triplet_output_file_3,
    text_format=triplet_text_format,
    skip_lines=triplet_skip_lines,
)

In [22]:
count = 0
for v in entity_dict.values():
    count += v is None

In [23]:
count

42169200

In [20]:
len(entity_dict)

50229038

In [21]:
with open(edge_output_file, 'w') as file_out:
    final_rel_mapping = {}
    for i, key in enumerate(edge_dict.keys():
        final_rel_mapping[i] = key
        file_out.write("{}\t{}\n".format(i, key))

In [22]:
with open(entity_output_file, 'w') as file_out:
    final_ent_mapping = {}
    for i, val in enumerate(entity_dict.keys()):
        final_ent_mapping[i] = key
        file_out.write("{}\t{}\n".format(i, key))

In [23]:
final_ent_mapping

{0: 31,
 1: 1088364,
 2: 3247091,
 3: 1308013,
 4: 7112200,
 5: 4916,
 6: 232415,
 7: 7021332,
 8: 1061257,
 9: 25648793,
 10: 1378312,
 11: 1115035,
 12: 213107,
 13: 128267,
 14: 41614,
 15: 223933,
 16: 1160895,
 17: 32,
 18: 38,
 19: 183,
 20: 347,
 21: 408,
 22: 212,
 23: 1246,
 24: 142,
 25: 145,
 26: 16,
 27: 252,
 28: 35,
 29: 43,
 30: 55,
 31: 77,
 32: 833,
 33: 843,
 34: 865,
 35: 96,
 36: 974,
 37: 30,
 38: 159,
 39: 668,
 40: 41,
 41: 148,
 42: 230,
 43: 801,
 44: 29999,
 45: 28,
 46: 1137059,
 47: 205317,
 48: 7411,
 49: 150,
 50: 188,
 51: 46,
 52: 239,
 53: 199614,
 54: 4366768,
 55: 155004,
 56: 12971,
 57: 12967,
 58: 55008046,
 59: 12973,
 60: 12976,
 61: 3911,
 62: 445553,
 63: 1079522,
 64: 7974978,
 65: 950958,
 66: 458,
 67: 1065,
 68: 7184,
 69: 41550,
 70: 8908,
 71: 13116,
 72: 42262,
 73: 7825,
 74: 141720,
 75: 1542735,
 76: 152299,
 77: 151991,
 78: 81299,
 79: 191384,
 80: 827525,
 81: 656801,
 82: 1043527,
 83: 899770,
 84: 340195,
 85: 188822,
 86: 782942

In [24]:
import pickle

In [25]:
folder = '../../AC297r_2019_Kensho/data/knowledge_graph_data/'

In [26]:
def write_dict_to_pickle(file, dictionary):
    with open(file, 'wb') as handle:
        pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
file_rel = folder + 'idx2id_edge_full.pickle'
write_dict_to_pickle(file_rel, final_rel_mapping)

In [28]:
file_ent = folder + 'idx2id_entity_full.pickle'
write_dict_to_pickle(file_ent, final_ent_mapping)

In [29]:
file_rel_text = folder + 'id2text_edge_full.pickle'
write_dict_to_pickle(file_rel_text, final_rel_dict)

In [30]:
file_ent_text = folder + 'id2text_entity_full.pickle'
write_dict_to_pickle(file_ent_text, final_ent_dict)

In [40]:
with open(file_rel_text, 'rb') as f:
    a = pickle.load(f)

In [41]:
a

{793: 'significant event',
 910: "topic's main category",
 527: 'has part',
 2670: 'has parts of the class',
 1419: 'shape',
 2184: 'history of topic',
 1552: 'has quality',
 398: 'child astronomical body',
 1343: 'described by source',
 828: 'has cause',
 1889: 'different from',
 2579: 'studied by',
 279: 'subclass of',
 17: 'country',
 131: 'located in the administrative territorial entity',
 163: 'flag',
 194: 'legislative body',
 190: 'twinned administrative body',
 138: 'named after',
 1792: 'category of associated people',
 421: 'located in time zone',
 1376: 'capital of',
 112: 'founded by',
 6: 'head of government',
 206: 'located in or next to body of water',
 47: 'shares border with',
 361: 'part of',
 5125: 'Wikimedia outline',
 1791: 'category of people buried here',
 1313: 'office held by head of government',
 1830: 'owner of',
 706: 'located on terrain feature',
 30: 'continent',
 610: 'highest point',
 1589: 'lowest point',
 31: 'instance of',
 35: 'head of state',
 37: 